In [1]:
import numpy as np
from numpy import linalg as LA
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from icecream import ic
from tqdm import trange
from numba import njit
import time

%load_ext snakeviz
# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [7]:
def plot_cameras_and_3D_points(X_est, X_opt, C_arr, axis_arr, s, path, save=False):
    
    fig = plt.figure(figsize=(8,6))
    ax = plt.axes(projection='3d')

    ax.plot(X_est[0], X_est[1], X_est[2], '.', ms=5, color='magenta', label='Noisy X')
    ax.plot(X_opt[0], X_opt[1], X_opt[2], '.', ms=1.7, color='lime', label='Opt. X')
    cv.plot_cameras_and_axes(ax, C_arr, axis_arr, s)

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.set_aspect('equal')
    ax.view_init(elev=-50, azim=-104, roll=20)
    ax.legend(loc="lower right")
    fig.tight_layout()
    if save:
        fig.savefig(path, dpi=300)
    plt.show()

In [3]:
def add_noise(pts, std):
    pts[:-1,:] += np.random.normal(0, std, pts[:-1,:].shape)
    return pts

In [4]:
path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-4'
data = r'\data-2023\data'
compex3 = r'\compEx3data.mat'
sift_matlab = r'\CE2_sift_matlab.mat'
report = r'\report-images'

eriks_sol = True

if eriks_sol:
    ind = 2

    x1 = cv.homogenize(cv.convert_mat_to_np(path+data+sift_matlab, 'xA'), multi=True)
    x2 = cv.homogenize(cv.convert_mat_to_np(path+data+sift_matlab, 'xB'), multi=True)
    inliers = np.load(path+data+'/CE2_inliers_{}.npy'.format(ind))
    x1 = x1[:,inliers]
    x2 = x2[:,inliers]
    X_est = np.load(path+data+'/CE2_X_valid_{}.npy'.format(ind))

    K = cv.convert_mat_to_np(path+data+'/compEx2data.mat', 'K')
    P1 = cv.transform(K, cv.get_canonical_camera())
    P2 = cv.transform(K, np.load(path+data+'/CE2_P2_valid_{}.npy'.format(ind)))
else:
    ind = 3
    
    x = cv.convert_mat_to_np(path+data+compex3, 'x')
    x1 = x[0,0]
    x2 = x[0,1]
    X_est = cv.convert_mat_to_np(path+data+compex3, 'X')

    P = cv.convert_mat_to_np(path+data+compex3, 'P')
    P1 = P[0,0]
    P2 = P[0,1]

stds = np.array([[0, 0],
                 [0, 3],
                 [0.1, 0],
                 [0.1, 3]])

X_noise_arr = []
X_opt_arr = []
mu = 1
n_its = 100

for std in stds:

    X_noise = add_noise(X_est, std[0])
    x1_noise = add_noise(x1, std[1])
    x2_noise = add_noise(x2, std[1])
    
    mu = 1
    n_its = 100

    print('\nstds', std)
    reproj_err_tot_est, _ = cv.compute_total_reprojection_error(P1, P2, X_noise, x1_noise, x2_noise)
    X_opt = cv.optimize_X(P1, P2, X_noise, x1_noise, x2_noise, mu, n_its)
    reproj_err_tot_opt, _ = cv.compute_total_reprojection_error(P1, P2, X_opt, x1_noise, x2_noise) 

    X_noise_arr.append(X_noise)
    X_opt_arr.append(X_opt)

X_noise_arr = np.concatenate(np.array(X_noise_arr)[:,np.newaxis], 0)
X_opt_arr = np.concatenate(np.array(X_opt_arr)[:,np.newaxis], 0)


stds [0. 0.]

Total reprojection error: 3420.75
Median reprojection error: 1.5
Avg. reprojection error: 1.86


100%|██████████| 1840/1840 [00:03<00:00, 533.39it/s]



Avg its: 1.9945652173913044
Max its: 2
Min its: 1

Total reprojection error: 3250.65
Median reprojection error: 1.44
Avg. reprojection error: 1.77

stds [0. 3.]

Total reprojection error: 69302.93
Median reprojection error: 31.31
Avg. reprojection error: 37.66


100%|██████████| 1840/1840 [00:04<00:00, 449.31it/s]



Avg its: 2.226086956521739
Max its: 3
Min its: 2

Total reprojection error: 20103.96
Median reprojection error: 4.85
Avg. reprojection error: 10.93

stds [0.1 0. ]

Total reprojection error: 713674263.63
Median reprojection error: 260373.39
Avg. reprojection error: 387866.45


100%|██████████| 1840/1840 [00:06<00:00, 275.72it/s]



Avg its: 3.7722826086956522
Max its: 5
Min its: 3

Total reprojection error: 20103.95
Median reprojection error: 4.85
Avg. reprojection error: 10.93

stds [0.1 3. ]

Total reprojection error: 735375861.66
Median reprojection error: 256616.18
Avg. reprojection error: 399660.79


100%|██████████| 1840/1840 [00:07<00:00, 253.83it/s]



Avg its: 3.7385869565217393
Max its: 5
Min its: 3

Total reprojection error: 38820.97
Median reprojection error: 9.24
Avg. reprojection error: 21.1


In [11]:
save = False

i = 3
X_noise = X_noise_arr[i]
X_opt = X_opt_arr[i]

P_arr = np.array([P1, P2])
C_arr, axis_arr = cv.compute_camera_and_normalized_principal_axis(P_arr, multi=True)
s = 1

plot_cameras_and_3D_points(X_noise, X_opt, C_arr, axis_arr, s, path+report+'/CE4_3D_{}_{}.png'.format(ind, i), save)